## This Jupyter Notebook contains basic functions for the implementation of algorithms in the paper [_The Development and Deployment of a Model for Hospital-level COVID-19 Associated Patient Demand Intervals from Consistent Estimators (DICE)_](https://arxiv.org/abs/2011.09377)

In [ ]:
# Libraries
library(ggplot2)
library(dplyr)
library(hrbrthemes)
library(changepoint)
Sys.setenv(LANGUAGE = "en")
library(reshape2)
library(forecast)
library(gridExtra)
library(invgamma)
rm(list=ls())
#work_dir='...'
#setwd(work_dir)

In [ ]:
# Actual hospital-level hospitalizations
shc.data=read.csv('...')

# Actual county-level hospitalizations
scc.data=read.csv('...')

# county-level hospitalization forecasts
county.forecast=read.csv('...')


N_list = scc.data$Hospitalizations
A_list = shc.data$covid_AAU
B_list = shc.data$covid_ICU

hat_p = sum(A_list)/sum(N_list)
hat_q = sum(B_list)/sum(N_list)

# Perfect Forcasts: Bootstrapping intervals

In [ ]:
bootstrap_perfect_forecast<-function(n,delta,alpha,b0,hat_p,hat_q,F_list,F_r){
    N_lambda=F_list
    A_lower_plugin_list=c()
    A_upper_plugin_list=c()
        
    B_lower_plugin_list=c()
    B_upper_plugin_list=c()  
    for(b in 1:b0){
        A_star_s=c()
        B_star_s=c()
        N_star_s=c()

    
         N_star_s=rpois(n=length(N_lambda),N_lambda)
        for(j in 1:n){
            multi_result = sapply(N_star_s[j], rmultinom,n=1, prob=c(hat_p,hat_q,(1-hat_p-hat_q)))
            A_j = multi_result[1,]
            B_j = multi_result[2,]
            A_star_s=c(A_star_s,A_j)
            B_star_s=c(B_star_s,B_j)           
        }
        p_star = sum(A_star_s)/sum(N_star_s)
        q_star = sum(B_star_s)/sum(N_star_s)
        
        A_lower_plugin_b=qpois(delta/2,p_star*F_r)
        A_upper_plugin_b=qpois(1-delta/2,p_star*F_r)
        
        B_lower_plugin_b=qpois(delta/2,q_star*F_r)
        B_upper_plugin_b=qpois(1-delta/2,q_star*F_r)
        
        A_lower_plugin_list=c(A_lower_plugin_list,A_lower_plugin_b)
        A_upper_plugin_list=c(A_upper_plugin_list,A_upper_plugin_b)
        
        B_lower_plugin_list=c(B_lower_plugin_list,B_lower_plugin_b)
        B_upper_plugin_list=c(B_upper_plugin_list,B_upper_plugin_b)        
    }
    A_lower_plugin=qpois(delta/2,hat_p*F_r)
    A_upper_plugin=qpois(1-delta/2,hat_p*F_r)
    B_lower_plugin=qpois(delta/2,hat_q*F_r)
    B_upper_plugin=qpois(1-delta/2,hat_q*F_r)
    
    z_al=ceiling(quantile(A_lower_plugin_list-A_lower_plugin,1-alpha))
    z_bl=ceiling(quantile(B_lower_plugin_list-B_lower_plugin,1-alpha))
    
    z_ar=floor(quantile(A_upper_plugin_list-A_upper_plugin,alpha))
    z_br=floor(quantile(B_upper_plugin_list-B_upper_plugin,alpha))
    
    A_lower_bootstrap=A_lower_plugin-z_al
    B_lower_bootstrap=B_lower_plugin-z_bl
    A_upper_bootstrap=A_upper_plugin-z_ar
    B_upper_bootstrap=B_upper_plugin-z_br
    
    return(list(c(A_lower_plugin,A_upper_plugin,B_lower_plugin,B_upper_plugin),
               c(A_lower_bootstrap,A_upper_bootstrap,B_lower_bootstrap,B_upper_bootstrap),
               A_lower_plugin_list,A_upper_plugin_list,B_lower_plugin_list,B_upper_plugin_list))
}

# Unbiased Forecasts with Lognormal Errors

In [ ]:
bootstrap_unbiased_forecast<-function(n,delta,alpha,b0,m,hat_p,hat_q,hat_mu,hat_sigma2,hat_rho,F_list,F_r){
    A_lower_plugin_list=c()
    A_upper_plugin_list=c()
        
    B_lower_plugin_list=c()
    B_upper_plugin_list=c()  
    for(b in 1:b0){
        A_star_s=c()
        B_star_s=c()
        N_star_s=c()
        Y=rep(0,n)
        Y[1]=rnorm(n=1,mean=hat_mu/(1-hat_rho),sd=sqrt(hat_sigma2/(1-hat_rho^2)))
        for(j in 2:n){
            Y[j]=hat_rho*Y[j-1]+rnorm(n=1,mean=hat_mu,sd=sqrt(hat_sigma2))      
        }
        N_star_s=rpois(n=n,F_list*exp(Y))
        fraction1 = mean((N_star_s^2-N_star_s)/F_list^2)
        fraction2 = mean(N_star_s[1:(length(N_star_s)-1)]*N_star_s[2:length(N_star_s)]/(F_list[1:(length(F_list)-1)]*F_list[2:length(F_list)]))

        fn <- function(param){
            rho=param[1]
            sigma2=param[2]
            mu=-sigma2/(2+2*rho)
            obj = 0
            obj = obj+(fraction1-exp(2*mu/(1-rho)+2*sigma2/(1-rho^2)))^2
            obj = obj+(fraction2-exp(2*mu/(1-rho)+sigma2/(1-rho)))^2
            obj = obj+10000*max(0,(rho^2-1))+10000*max(0,(-sigma2))
            return(obj)
        }
        result = optim(rep(0,2),lower=c(-0.99,0),upper=c(0.99,1),fn,method="L-BFGS-B")
        hat_rho_star=result$par[1]
        hat_sigma2_star=result$par[2]
        hat_mu_star=-hat_sigma2_star/(2+2*hat_rho_star^2)
        
        for(j in 1:n){
            multi_result = sapply(N_star_s[j], rmultinom,n=1, prob=c(hat_p,hat_q,(1-hat_p-hat_q)))
            A_j = multi_result[1,]
            B_j = multi_result[2,]
            A_star_s=c(A_star_s,A_j)
            B_star_s=c(B_star_s,B_j)           
        }
        p_star = sum(A_star_s)/sum(N_star_s)
        q_star = sum(B_star_s)/sum(N_star_s)
        
        A_r_star=rep(0,m)
        B_r_star=rep(0,m)
        for(i in 1:m){
            Yr=rnorm(n=1,mean=hat_mu_star/(1-hat_rho_star),
                     sd=sqrt(hat_sigma2_star/(1-hat_rho_star^2)))
            A_r_star[i]=rpois(n=1,lambda=p_star*F_r*exp(Yr))
            B_r_star[i]=rpois(n=1,lambda=q_star*F_r*exp(Yr))
        }
               
        A_lower_plugin_b=ceiling(quantile(A_r_star,delta/2))
        A_upper_plugin_b=floor(quantile(A_r_star,1-delta/2))
        
        B_lower_plugin_b=ceiling(quantile(B_r_star,delta/2))
        B_upper_plugin_b=floor(quantile(B_r_star,1-delta/2))
        
        A_lower_plugin_list=c(A_lower_plugin_list,A_lower_plugin_b)
        A_upper_plugin_list=c(A_upper_plugin_list,A_upper_plugin_b)
        
        B_lower_plugin_list=c(B_lower_plugin_list,B_lower_plugin_b)
        B_upper_plugin_list=c(B_upper_plugin_list,B_upper_plugin_b)        
    }
    A_r=rep(0,m)
    B_r=rep(0,m)
    for(i in 1:m){
        Yr=rnorm(n=1,mean=hat_mu/(1-hat_rho),
                     sd=sqrt(hat_sigma2/(1-hat_rho^2)))
        A_r[i]=rpois(n=1,lambda=hat_p*F_r*exp(Yr))
        B_r[i]=rpois(n=1,lambda=hat_q*F_r*exp(Yr))
    }
               
    
    A_lower_plugin=ceiling(quantile(A_r,delta/2))
    A_upper_plugin=floor(quantile(A_r,1-delta/2))
    B_lower_plugin=ceiling(quantile(B_r,delta/2))
    B_upper_plugin=floor(quantile(B_r,1-delta/2))

    
    z_al=ceiling(quantile(A_lower_plugin_list-A_lower_plugin,1-alpha))
    z_bl=ceiling(quantile(B_lower_plugin_list-B_lower_plugin,1-alpha))
    
    z_ar=floor(quantile(A_upper_plugin_list-A_upper_plugin,alpha))
    z_br=floor(quantile(B_upper_plugin_list-B_upper_plugin,alpha))
    
    A_lower_bootstrap=A_lower_plugin-z_al
    B_lower_bootstrap=B_lower_plugin-z_bl
    A_upper_bootstrap=A_upper_plugin-z_ar
    B_upper_bootstrap=B_upper_plugin-z_br
    
    return(list(c(A_lower_plugin,A_upper_plugin,B_lower_plugin,B_upper_plugin),
               c(A_lower_bootstrap,A_upper_bootstrap,B_lower_bootstrap,B_upper_bootstrap),
               A_lower_plugin_list,A_upper_plugin_list,B_lower_plugin_list,B_upper_plugin_list))
}

# Biased Forecasts with Lognormal Errors

In [ ]:
bootstrap_biased_forecast <- function(n,delta,alpha,b0,m,hat_p,hat_q,hat_mu,hat_sigma2,hat_rho,F_list,F_r){
    A_lower_plugin_list=c()
    A_upper_plugin_list=c()
        
    B_lower_plugin_list=c()
    B_upper_plugin_list=c()  
    for(b in 1:b0){
        A_star_s=c()
        B_star_s=c()
        N_star_s=c()
        Y=rep(0,n)
        Y[1]=rnorm(n=1,mean=hat_mu/(1-hat_rho),sd=sqrt(hat_sigma2/(1-hat_rho^2)))
        for(j in 2:n){
            Y[j]=hat_rho*Y[j-1]+rnorm(n=1,mean=hat_mu,sd=sqrt(hat_sigma2))      
        }
        N_star_s=rpois(n=n,F_list*exp(Y))
        
        fraction1 = mean(N_star_s/F_list)
        fraction2 = mean((N_star_s^2-N_star_s)/F_list^2)
        fraction3 = mean(N_star_s[1:(length(N_star_s)-1)]*N_star_s[2:length(N_star_s)]/(F_list[1:(length(F_list)-1)]*F_list[2:length(F_list)]))

        fn <- function(param){
            rho=param[1]
            sigma2=param[2]
            mu=param[3]
            obj = 0
            obj = obj+(fraction1-exp(mu/(1-rho)+0.5*sigma2/(1-rho^2)))^2
            obj = obj+(fraction2-exp(2*mu/(1-rho)+2*sigma2/(1-rho^2)))^2
            obj = obj+(fraction3-exp(2*mu/(1-rho)+sigma2/(1-rho)))^2
            obj = obj+10000*max(0,(rho^2-1))+10000*max(0,(-sigma2))
            return(obj)
        }
        result = optim(rep(0,3),lower=c(-0.99,0.0001,-4),upper=c(0.99,4,4),fn,method="L-BFGS-B")

        hat_rho_star=result$par[1]
        hat_sigma2_star=result$par[2]
        hat_mu_star=result$par[3]
        
        for(j in 1:n){
            multi_result = sapply(N_star_s[j], rmultinom,n=1, prob=c(hat_p,hat_q,(1-hat_p-hat_q)))
            A_j = multi_result[1,]
            B_j = multi_result[2,]
            A_star_s=c(A_star_s,A_j)
            B_star_s=c(B_star_s,B_j)           
        }
        p_star = sum(A_star_s)/sum(N_star_s)
        q_star = sum(B_star_s)/sum(N_star_s)
        
        A_r_star=rep(0,m)
        B_r_star=rep(0,m)
        for(i in 1:m){
            Yr=rnorm(n=1,mean=hat_mu_star/(1-hat_rho_star),
                     sd=sqrt(hat_sigma2_star/(1-hat_rho_star^2)))
            A_r_star[i]=rpois(n=1,lambda=p_star*F_r*exp(Yr))
            B_r_star[i]=rpois(n=1,lambda=q_star*F_r*exp(Yr))
        }
               
        A_lower_plugin_b=ceiling(quantile(A_r_star,delta/2))
        A_upper_plugin_b=floor(quantile(A_r_star,1-delta/2))
        
        B_lower_plugin_b=ceiling(quantile(B_r_star,delta/2))
        B_upper_plugin_b=floor(quantile(B_r_star,1-delta/2))
        
        A_lower_plugin_list=c(A_lower_plugin_list,A_lower_plugin_b)
        A_upper_plugin_list=c(A_upper_plugin_list,A_upper_plugin_b)
        
        B_lower_plugin_list=c(B_lower_plugin_list,B_lower_plugin_b)
        B_upper_plugin_list=c(B_upper_plugin_list,B_upper_plugin_b)        
    }
    A_r=rep(0,m)
    B_r=rep(0,m)
    for(i in 1:m){
        Yr=rnorm(n=1,mean=hat_mu/(1-hat_rho),
                     sd=sqrt(hat_sigma2/(1-hat_rho^2)))
        A_r[i]=rpois(n=1,lambda=hat_p*F_r*exp(Yr))
        B_r[i]=rpois(n=1,lambda=hat_q*F_r*exp(Yr))
    }
               
    
    A_lower_plugin=ceiling(quantile(A_r,delta/2))
    A_upper_plugin=floor(quantile(A_r,1-delta/2))
    B_lower_plugin=ceiling(quantile(B_r,delta/2))
    B_upper_plugin=floor(quantile(B_r,1-delta/2))

    
    z_al=ceiling(quantile(A_lower_plugin_list-A_lower_plugin,1-alpha))
    z_bl=ceiling(quantile(B_lower_plugin_list-B_lower_plugin,1-alpha))
    
    z_ar=floor(quantile(A_upper_plugin_list-A_upper_plugin,alpha))
    z_br=floor(quantile(B_upper_plugin_list-B_upper_plugin,alpha))
    
    A_lower_bootstrap=A_lower_plugin-z_al
    B_lower_bootstrap=B_lower_plugin-z_bl
    A_upper_bootstrap=A_upper_plugin-z_ar
    B_upper_bootstrap=B_upper_plugin-z_br
    
    return(list(c(A_lower_plugin,A_upper_plugin,B_lower_plugin,B_upper_plugin),
               c(A_lower_bootstrap,A_upper_bootstrap,B_lower_bootstrap,B_upper_bootstrap),
               A_lower_plugin_list,A_upper_plugin_list,B_lower_plugin_list,B_upper_plugin_list))
}